In [1]:
%run preamble_scripts.py

# Simulation study: the well specified case 500 observations

In [2]:
n_sim, nobs, expo = 100, 500, 250
err_pnl, popsize = 3e5, 5000

In [3]:
# Setting the true model
f1, f2 = bs.loss_model("Weibull", ["k1", "β1"]), bs.loss_model("Lomax", ["α2", "σ2"])
f_true = bs.spliced_loss_model(f1, f2, "continuous")
parms_true = np.array([1/2, 1, 2.5, 3, 1.5])

f_true.set_ppf(), f_true.set_pdf(),f_true.set_cdf()

# We set the priority to the 90% quantile and the limit to the 0.99% quantile
P, L = f_true.ppf(parms_true, 0.9), f_true.ppf(parms_true, 0.99)
premiums = f_true.PP(parms_true), f_true.XOLP(parms_true, P, L)

%time PnLs = np.array(f_true.PnL(parms_true, P, L, expo, premiums, safety_loadings = [0.05, 0.05],  n_sim = err_pnl))
true_VaRs = [f_true.ppf(parms_true, prob) for prob in [0.95, 0.99, 0.995]]
true_cap = np.quantile(PnLs, [0.005, 0.01, 0.05])
true_cap

CPU times: user 39.6 ms, sys: 0 ns, total: 39.6 ms
Wall time: 39.6 ms


array([-433.78618324, -119.00500314,  -52.95173865])

We are interested in the estimations of the extreme quantile of the claim size distribution (of order 0.95, 0.99, 0.995) and the quantile of the aggregate losses over one year with a XOL reinsurance agreement

In [6]:
Xs = [f_true.sample(parms_true, nobs) for k in range(n_sim)]
def fit_one(k, f, f_name,  prior, X):
    trace, log_marg, DIC, WAIC = bs.smc(X, f, popsize, prior, verbose = False)
    PnLs = np.array(f.PnL(trace.mean().values, P, L, expo, premiums, safety_loadings = [0.05, 0.05],  n_sim = err_pnl))
    cap = np.quantile(PnLs, [0.005, 0.01, 0.05])
    return([k, f_name , nobs, log_marg, trace["γ"].mean(), parms_true[-1] ] + cap.tolist() + true_cap.tolist())
def fit_all(k):
    print("simulation #"+ str(k))
    col_names = ["sim", "model_name", "nobs", "log_marg", "γ_map", "γ_true", "cap005", "cap01", "cap05", "cap005_true", "cap01_true", "cap05_true" ]
    df_temp = pd.DataFrame([fit_one(k, fs[i], f_names[i], prior_spliced_model[i], Xs[k]) for i in range(len(fs))], columns = col_names)
    df_temp['posterior_probability'] = np.exp(df_temp['log_marg'].values - np.max(df_temp['log_marg'].values)) / \
    np.sum(np.exp(df_temp['log_marg'].values - np.max(df_temp['log_marg'].values)))  
    γ_BMA = np.dot(df_temp['posterior_probability'].values, df_temp['γ_map'])
    cap_bma = np.dot(df_temp['posterior_probability'].values, df_temp[["cap005", "cap01", "cap05"]].values)
    BMA_df = pd.DataFrame([[k, "BMA" , nobs, None, γ_BMA, parms_true[-1] ] + cap_bma.tolist() + true_cap.tolist()],
            columns = col_names)
    df_temp = pd.concat([df_temp, BMA_df])
    return(df_temp)
%time res = Parallel(n_jobs= 50)(delayed(fit_all)(k) for k in range(n_sim))
pd.concat(res).to_csv("../../Data/Simulations/simu_wellspec_"+str(nobs)+".csv", sep=',')

CPU times: user 130 ms, sys: 12.2 ms, total: 142 ms
Wall time: 34.8 s


In [ ]:
pd.concat(res)